In [1]:
from os import listdir
import numpy as np
import pandas as pd
import string
from nltk.translate.bleu_score import corpus_bleu
from pickle import load, dump
from IPython.display import Image
import os
 
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import plot_model, to_categorical
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.vgg16 import VGG16, preprocess_input 
from keras.models import Model
from keras.layers import Input, Dense, Flatten, LSTM
from keras.layers import RepeatVector, TimeDistributed
from keras.layers import Embedding
from keras.layers.merge import concatenate
from keras.layers.pooling import GlobalMaxPooling2D
from keras.layers.pooling import GlobalAveragePooling2D

from caption_generation_model_API import *

C:\Anaconda\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
path = 'D:/Program/dataset/Flickr8K/'

# load dev set
filename = path + 'Flickr8k_text/Flickr_8k.trainImages.txt'
dataset = load_set(filename)
print('Dataset: {}'.format(len(dataset)))
# train-test split
train, test = train_test_split(dataset)
# descriptions
train_descriptions = load_clean_descriptions(path+'descriptions.txt', train)
test_descriptions = load_clean_descriptions(path+'descriptions.txt', test)
print('Descriptions: train={}, test={}'.format(len(train_descriptions), len(test_descriptions)))
# photo features
train_features = load_photo_features(path+'features.pkl', train)
test_features = load_photo_features(path+'features.pkl', test)
print('Photos: train={}, test={}'.format(len(train_features), len(test_features)))
# prepare tokenizer
tokenizer = create_tokenizer(train_descriptions)
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: {}'.format(vocab_size))
# determine the maximum sequence length
max_length = max(len(s.split()) for s in list(train_descriptions.values()))
print('Description Length: {}'.format(max_length))

Dataset: 6000
Descriptions: train=5500, test=500
Photos: train=5500, test=500
Vocabulary Size: 3699
Description Length: 30


In [3]:
verbose = 1
n_epochs = 500
n_photos_per_update = 2
n_batches_per_epoch = int(len(train) / n_photos_per_update)
n_repeats = 1

In [4]:
def evaluate_and_print(model, descriptions, photos, tokenizer, max_length):
    actual, predicted = list(), list()
    # step over the whole set
    for key, desc in descriptions.items():
        # generate description
        yhat = generate_desc(model, tokenizer, photos[key], max_length)
        # store actual and predicted
        actual.append([desc.split()])
        predicted.append(yhat.split())
        print('Actual:    {}'.format(desc))
        print('Predicted: {}'.format(yhat))
        print()
        if len(actual) >= 50:
            break
    # calculate BLEU score
    bleu = corpus_bleu(actual, predicted)
    return bleu

In [5]:
from keras.models import load_model

In [6]:
model_path = "D:/Program/train_model/image_caption/fe_avg_pool/fe_avg_pool.h5"
model = load_model(model_path)

In [7]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 7, 7, 512)    0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 30)           0                                            
__________________________________________________________________________________________________
global_average_pooling2d_2 (Glo (None, 512)          0           input_3[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 30, 50)       184950      input_4[0][0]                    
__________________________________________________________________________________________________
dense_5 (D

In [8]:
trainset_score = evaluate_and_print(model, train_descriptions, train_features, tokenizer, max_length)
trainset_score

Actual:    startseq child in pink dress is climbing up set of stairs in an entry way endseq
Predicted: startseq blue of people in black fall hovering endseq

Actual:    startseq black dog and spotted dog are fighting endseq
Predicted: startseq black and white dog is running on over endseq

Actual:    startseq little girl covered in paint sits in front of painted rainbow with her hands in bowl endseq
Predicted: startseq boy is riding on the stands endseq

Actual:    startseq man lays on bench while his dog sits by him endseq
Predicted: startseq man in red shirt is snowy pants on the shore endseq

Actual:    startseq man in an orange hat starring at something endseq
Predicted: startseq man in group shirt and each and half hat and hat is smiling endseq

Actual:    startseq child playing on rope net endseq
Predicted: startseq blue of runs are running on the waterskies picture endseq

Actual:    startseq black and white dog is running in grassy garden surrounded by white fence endseq
Predic

0.07507702587419578